In [11]:
import numpy as np

class HillCipher:
    def __init__(self, key):
        self.key = key
        self.key_matrix = self.__key_to_matrix(key)
        self.key_inverse = self.__inverse_key(self.key_matrix)
        self.alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    def __key_to_matrix(self, key):
        key_size = int(np.sqrt(len(key)))
        key_matrix = np.zeros((key_size, key_size))
        for i in range(key_size):
            for j in range(key_size):
                key_matrix[i][j] = ord(key[i * key_size + j]) - 65
        return key_matrix

    def __inverse_key(self, key_matrix):
        key_matrix = np.matrix(key_matrix)
        determinant = int(np.round(np.linalg.det(key_matrix)))
        determinant_inverse = self.__multiplicative_inverse(determinant)

        key_matrix_inverse = np.round(
            np.matrix(np.linalg.inv(key_matrix) * determinant * determinant_inverse)
        ).astype(int)

        return key_matrix_inverse

    def __multiplicative_inverse(self, a, m=26):
        a = a % m
        for i in range(1, m):
            if (a * i) % m == 1:
                return i
        return 1

    def encrypt(self, plaintext):
        plaintext = plaintext.upper()
        plaintext_size = len(plaintext)
        plaintext_matrix = np.zeros((plaintext_size // 2 + plaintext_size % 2, 2))

        for i in range(plaintext_size // 2):
            plaintext_matrix[i][0] = ord(plaintext[i * 2]) - 65
            plaintext_matrix[i][1] = ord(plaintext[i * 2 + 1]) - 65

        if plaintext_size % 2 == 1:
            plaintext_matrix[-1][0] = ord(plaintext[-1]) - 65
        
        print(plaintext_matrix)
        print(self.key_matrix)
        ciphertext_matrix = np.dot(plaintext_matrix, self.key_matrix)
        ciphertext_matrix = ciphertext_matrix % 26

        ciphertext = ""
        for i in range(ciphertext_matrix.shape[0]):
            ciphertext += self.alphabet[ciphertext_matrix[i][0]]
            ciphertext += self.alphabet[ciphertext_matrix[i][1]]

        return ciphertext

    def decrypt(self, ciphertext):
        ciphertext = ciphertext.upper()
        ciphertext_size = len(ciphertext)
        ciphertext_matrix = np.zeros((ciphertext_size // 2, 2))

        for i in range(ciphertext_size // 2):
            ciphertext_matrix[i][0] = ord(ciphertext[i * 2]) - 65
            ciphertext_matrix[i][1] = ord(ciphertext[i * 2 + 1]) - 65

        plaintext_matrix = np.dot(ciphertext_matrix, self.key_inverse)
        plaintext_matrix = plaintext_matrix % 26

        plaintext = ""
        for i in range(plaintext_matrix.shape[0]):
            plaintext += self.alphabet[plaintext_matrix[i][0]]
            plaintext += self.alphabet[plaintext_matrix[i][1]]

        return plaintext


In [16]:
# Example usage
key = "GiweURP"
plaintext = "2yeO"

hill_cipher = HillCipher(key)

# Encrypt the plaintext
ciphertext = hill_cipher.encrypt(plaintext)
print(f"Plaintext: {plaintext}")
print(f"Ciphertext: {ciphertext}")

# Decrypt the ciphertext
decrypted_plaintext = hill_cipher.decrypt(ciphertext)
print(decrypted_plaintext)


[[  7.   4.]
 [ 11.  11.]
 [-15.  -9.]
 [-16. -14.]
 [-10. -11.]
 [-15.  24.]
 [  4.  14.]]
[[ 6. 24.  1.]
 [13. 16. 10.]
 [48. 40. 54.]]


ValueError: shapes (7,2) and (3,3) not aligned: 2 (dim 1) != 3 (dim 0)

In [2]:

def ecb_encrypt(plaintext, key):
    block_size = len(key)
    C=make_key(key)
    ciphertext = ''
    for i in range(0, len(plaintext), block_size):
        block = plaintext[i:i+block_size]
        ciphertext += encrypt(block,C)
    return ciphertext

def ecb_decrypt(ciphertext, key):
    
    block_size = len(key)
    plaintext = ''
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i+block_size]
        Akey=make_key(key)
        plaintext += decrypt(block,Akey)
    return plaintext

import numpy as np


def encrypt(msg,C):
    msg = msg.replace(" ", "")
    len_check = len(msg) % 2 == 0
    if not len_check:
        msg += "0"
    P = create_matrix_of_integers_from_string(msg)
    msg_len = int(len(msg) / 2)
    encrypted_msg = ""

    for i in range(msg_len):
        # Dot product
        
        row_0 = P[0][i] * C[0][0] + P[1][i] * C[0][1]
        integer = int(row_0 % 26 + 65)
        encrypted_msg += chr(integer)
        row_1 = P[0][i] * C[1][0] + P[1][i] * C[1][1]
        integer = int(row_1 % 26 + 65)
        encrypted_msg += chr(integer)
    return encrypted_msg

def decrypt(encrypted_msg,C):
    determinant = C[0][0] * C[1][1] - C[0][1] * C[1][0]
    determinant = determinant % 26
    multiplicative_inverse = find_multiplicative_inverse(determinant)
    C_inverse = C
    C_inverse[0][0], C_inverse[1][1] = C_inverse[1, 1], C_inverse[0, 0]
    C[0][1] *= -1
    C[1][0] *= -1
    for row in range(2):
        for column in range(2):
            C_inverse[row][column] *= multiplicative_inverse
            C_inverse[row][column] = C_inverse[row][column] % 26

    P = create_matrix_of_integers_from_string(encrypted_msg)
    msg_len = int(len(encrypted_msg) / 2)
    decrypted_msg = ""
    for i in range(msg_len):
        column_0 = P[0][i] * C_inverse[0][0] + P[1][i] * C_inverse[0][1]
        integer = int(column_0 % 26 + 65)
        decrypted_msg += chr(integer)
        column_1 = P[0][i] * C_inverse[1][0] + P[1][i] * C_inverse[1][1]
        integer = int(column_1 % 26 + 65)
        decrypted_msg += chr(integer)
    if decrypted_msg[-1] == "0":
        decrypted_msg = decrypted_msg[:-1]
    return decrypted_msg

def find_multiplicative_inverse(determinant):
    multiplicative_inverse = -1
    for i in range(26):
        inverse = determinant * i
        if inverse % 26 == 1:
            multiplicative_inverse = i
            break
    return multiplicative_inverse


def make_key(cipher):
    determinant = 0
    C = None
    while True:
        C = create_matrix_of_integers_from_string(cipher)
        determinant = C[0][0] * C[1][1] - C[0][1] * C[1][0]
        determinant = determinant % 26
        
        inverse_element = find_multiplicative_inverse(determinant)
        
        if inverse_element == -1:
            print("Determinant is not relatively prime to 26, uninvertible key")
        elif np.amax(C) > 26 and np.amin(C) < 0:
            print("Only a-z characters are accepted")
            print(np.amax(C), np.amin(C))
        else:
            break
    return C

def create_matrix_of_integers_from_string(string):
    integers = [chr_to_int(c) for c in string]
    length = len(integers)
    M = np.zeros((2, int(length / 2)), dtype=np.int32)
    iterator = 0
    for column in range(int(length / 2)):
        for row in range(2):
            M[row][column] = integers[iterator]
            iterator += 1
    return M

def chr_to_int(char):
    char = char.upper()
    integer = ord(char) - 65
    return integer


key = 'Hllo'
plaintext = 'MyNameIsAreej'
ciphertext = ecb_encrypt(plaintext, key)
print("PlainText : ",plaintext)
print("Encrypted ciphertext:", ciphertext)
decrypted_plaintext = ecb_decrypt(ciphertext, key)
print("Decrypted plaintext:", decrypted_plaintext)


PlainText :  MyNameIsAreej
Encrypted ciphertext: KANNYGUCFEUWGR
Decrypted plaintext: MYNAMEISAREEJJ
